# MAPC Team 1 Notebook

In [1]:
# code block for imports 
import pandas as pd 
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap


In [2]:
# functions for reading data as a df and inserting into sql db
def readData(filename):
    data = pd.read_csv(filename)
    return data

def insertDataToDB(tablename, df,conn):
    df.to_sql(tablename,conn, if_exists="replace")


In [3]:
# connect to database cs506MAPC.db
conn = sqlite3.connect('cs506MAPC.db')

# read data from FCC June 2019 data. 
# If this line causes errors for you, you first need to download the data and 
# and unzip it into your local copy of your repository. (Can't upload the CSV to git because it's too large)
# to download the zip file, go here --> https://github.com/MAPC/broadband-data-bu/tree/main/FCC%20data
# NOTE: Make sure that you have the .csv file in the 'MAPC Broadband Equity - Team 1' folder and that you rename it
# to "fcc_data_june2019.csv" 

df = readData("fcc_data_june2019.csv")
insertDataToDB("fcc_data_t",df,conn)

**Command to import cities with their long and lat**

In [4]:
df = readData("MACityLongLat.csv")
insertDataToDB("CityLongLat_t",df,conn)

In [5]:
qryStr = "CREATE TABLE IF NOT EXISTS censusblocks_t (town TEXT, blockcode TEXT PRIMARY KEY NOT NULL)"
cursor = conn.cursor()
cursor.execute(qryStr)

dict = {}


for filename in os.listdir('census_blocks'):
    readFile = os.path.join('census_blocks', filename)
    names = filename.split(".")
    cityName = names[0].upper()
    f = open(readFile,"r", encoding="latin-1");
    lines = f.readlines();
    count = 0
    for line in lines:
        if count == 0:
            count += 1
            continue
        entries = line.split(";")
        if len(entries) != 6:
            continue
        tracts = entries[3].split(".");
        censusblock = entries[1] + entries[2] + tracts[0] + tracts[1]
        
        if censusblock not in dict:
            dict[censusblock] = cityName
        else:
            continue
        cursor.execute("INSERT INTO censusblocks_t VALUES(?,?)",(cityName,censusblock));
        conn.commit()

IntegrityError: UNIQUE constraint failed: censusblocks_t.blockcode

In [ ]:
df = readData("median_income.csv")
print(df.head(10))
df_filtered = df[['ct10_id','mhi','mhi_me','o_mhi','o_mhi_me','r_mhi','r_mhi_me']]
insertDataToDB("median_income_t",df_filtered,conn)


In [ ]:
df = readData("race_ethnicity.csv")
df = df.fillna(0)
df_filtered = df[['ct10_id','nh_p','nhwhi_p','nhaa_p','nhna_p','nhas_p','nhpi_p','nhoth_p','nhmlt_p','lat_p']]
insertDataToDB("race_ethnicity_t",df_filtered,conn)


In [10]:
df = readData("demographic.csv")
df = df.fillna(0)
df_filtered = df[['bg10_id','pop','male','female','fam','hh']]
insertDataToDB("demographics_t",df_filtered,conn)


In [8]:
df = readData("laborForceData.csv")
df = df.fillna(0)
df_filtered = df[['bg10_id','emp','unemp','emp_p','unemp_p','lf']]
insertDataToDB("laborforce_t",df_filtered,conn)